# This notebook extract the informations regarding the first election.

This step extracts tables from Yahoo finance, creates and renames csv, that are finally downloaded.

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Liste des URLs et noms correspondants
urls = {
    "https://fr.finance.yahoo.com/quote/%5EFCHI/history/?period1=1338508800&period2=1370044800": "CAC40_O",
    "https://fr.finance.yahoo.com/quote/%5EGSPC/history/?period1=1338508800&period2=1370044800": "S&P500_O",
    "https://fr.finance.yahoo.com/quote/GC%3DF/history/?period1=1338508800&period2=1370044800": "Gold_O",
    "https://fr.finance.yahoo.com/quote/EURUSD%3DX/history/?period1=1338508800&period2=1370044800": "EURUSD_O",
    "https://fr.finance.yahoo.com/quote/%5EIXIC/history/?period1=1338508800&period2=1370044800": "NASDAQ_O",
    "https://fr.finance.yahoo.com/quote/%5EN225/history/?period1=1338508800&period2=1370044800": "Nikkei_O",
    "https://fr.finance.yahoo.com/quote/ZN%3DF/history/?period1=1338508800&period2=1370044800": "Taux10ans_O"
}

# Fonction pour scraper un tableau sur une URL
def scrape_table_from_url(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')

    if table:
        rows = table.find_all('tr')
        data = []

        for row in rows:
            cols = row.find_all(['th', 'td'])
            cols = [col.text.strip() for col in cols]
            data.append(cols)

        # Création d'un DataFrame pandas
        df = pd.DataFrame(data)
        return df
    else:
        print(f"Aucun tableau trouvé pour l'URL : {url}")
        return None

# Scraper tous les tableaux des URLs
all_tables = {}

for url in urls:
    df = scrape_table_from_url(url)
    if df is not None:
        all_tables[url] = df




# Scraper les tableaux et ajouter la colonne à gauche
for url, name in urls.items():
    df = scrape_table_from_url(url)
    if df is not None:
        # Ajouter la colonne avec le nom du marché à gauche
        df.insert(0, "Indice", name)
        
        # Sauvegarde du DataFrame dans un fichier CSV avec le nom spécifique
        filename = f"{name}_data.csv"
        df.to_csv(filename, index=False)
        print(f"Fichier CSV enregistré sous le nom : {filename}")


Fichier CSV enregistré sous le nom : CAC40_O_data.csv
Fichier CSV enregistré sous le nom : S&P500_O_data.csv
Fichier CSV enregistré sous le nom : Gold_O_data.csv
Fichier CSV enregistré sous le nom : EURUSD_O_data.csv
Fichier CSV enregistré sous le nom : NASDAQ_O_data.csv
Fichier CSV enregistré sous le nom : Nikkei_O_data.csv
Fichier CSV enregistré sous le nom : Taux10ans_O_data.csv


We transform the previously made CSV into pandas DataFrames

In [26]:
CAC = pd.read_csv('CAC40_O_data.csv')
SandP500 = pd.read_csv('S&P500_O_data.csv')
Gold = pd.read_csv('Gold_O_data.csv')
EURUSD = pd.read_csv('EURUSD_O_data.csv')
NASDAC = pd.read_csv('NASDAQ_O_data.csv')
Nikkei = pd.read_csv('Nikkei_O_data.csv')
Taux10ans = pd.read_csv('Taux10ans_O_data.csv')

Creating a list of DataFrames for merging each of them at once

In [27]:
# Liste des DataFrames déjà chargés
dfs = {
    "CAC": CAC,
    "SandP500": SandP500,
    "Gold": Gold,
    "EURUSD": EURUSD,
    "NASDAC": NASDAC,
    "Nikkei": Nikkei,
    "Taux10ans": Taux10ans
}

In [28]:
from datetime import datetime

# Date de l'élection de Donald Trump
date_election = datetime(2024, 7, 1)

# Concaténer tous les DataFrames en un seul
final_df = pd.concat(dfs.values(), ignore_index=True)


In [29]:
final_df

,Indice,0,1,2,3,4,5,6
0,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonctio...,Clôture ajustée Cours de clôture ajusté p...,Volume
1,CAC40_O,31 mai 2013,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800
2,CAC40_O,30 mai 2013,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200
3,CAC40_O,29 mai 2013,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600
4,CAC40_O,28 mai 2013,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800
...,...,...,...,...,...,...,...,...
1761,Taux10ans_O,7 juin 2012,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208
1762,Taux10ans_O,6 juin 2012,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137
1763,Taux10ans_O,5 juin 2012,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120
1764,Taux10ans_O,4 juin 2012,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905


Reseting the index to have the first row as column names.

In [30]:
if final_df.iloc[0].isna().sum() > 0:
    print("Attention : certaines colonnes de la première ligne sont vides.")
else:
    final_df.columns = final_df.iloc[0]
    final_df = final_df.drop(0).reset_index(drop=True)

In [31]:
final_df

,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonction des fractionnements.,Clôture ajustée Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.,Volume
0,CAC40_O,31 mai 2013,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800
1,CAC40_O,30 mai 2013,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200
2,CAC40_O,29 mai 2013,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600
3,CAC40_O,28 mai 2013,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800
4,CAC40_O,27 mai 2013,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400
...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,7 juin 2012,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208
1761,Taux10ans_O,6 juin 2012,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137
1762,Taux10ans_O,5 juin 2012,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120
1763,Taux10ans_O,4 juin 2012,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905


We change the date format 

In [32]:
import locale
locale.setlocale(locale.LC_TIME, 'fr_FR.UTF-8')
final_df['Date'] = pd.to_datetime(final_df['Date'], format='%d %b %Y', errors='coerce')


We create a after/before the election variable 

In [33]:

# Conversion des colonnes de date et ajout de la colonne "Avant/Après élection"
final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')
final_df['Avant_Apres_Election'] = final_df['Date'].apply(lambda x: 'Before' if x < date_election else 'After')

# Exporter vers un CSV consolidé
final_df.to_csv("indices_financiers_consolides.csv", index=False)
print("Le fichier CSV consolidé a été enregistré sous le nom : indices_financiers_consolides.csv")

Le fichier CSV consolidé a été enregistré sous le nom : indices_financiers_consolides.csv


In [34]:
final_df

,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonction des fractionnements.,Clôture ajustée Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.,Volume,Avant_Apres_Election
0,CAC40_O,2013-05-31,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800,Before
1,CAC40_O,2013-05-30,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200,Before
2,CAC40_O,2013-05-29,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600,Before
3,CAC40_O,2013-05-28,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800,Before
4,CAC40_O,2013-05-27,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400,Before
...,...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,2012-06-07,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208,Before
1761,Taux10ans_O,2012-06-06,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137,Before
1762,Taux10ans_O,2012-06-05,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120,Before
1763,Taux10ans_O,2012-06-04,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905,Before


Deleting empty date values

In [35]:
final_df = final_df.dropna(subset=['Date'])  # Supprime les lignes sans date
final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')  # Convertit la colonne en datetime


C:\Users\sarto\AppData\Local\Temp\ipykernel_4940\239324199.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')  # Convertit la colonne en datetime


In [36]:
final_df

,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonction des fractionnements.,Clôture ajustée Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.,Volume,Avant_Apres_Election
0,CAC40_O,2013-05-31,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800,Before
1,CAC40_O,2013-05-30,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200,Before
2,CAC40_O,2013-05-29,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600,Before
3,CAC40_O,2013-05-28,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800,Before
4,CAC40_O,2013-05-27,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400,Before
...,...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,2012-06-07,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208,Before
1761,Taux10ans_O,2012-06-06,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137,Before
1762,Taux10ans_O,2012-06-05,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120,Before
1763,Taux10ans_O,2012-06-04,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905,Before


Some verifications

In [37]:
filtered_df_before_election = final_df[final_df['Avant_Apres_Election'] == 'Before']
filtered_df_before_election


,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonction des fractionnements.,Clôture ajustée Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.,Volume,Avant_Apres_Election
0,CAC40_O,2013-05-31,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800,Before
1,CAC40_O,2013-05-30,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200,Before
2,CAC40_O,2013-05-29,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600,Before
3,CAC40_O,2013-05-28,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800,Before
4,CAC40_O,2013-05-27,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400,Before
...,...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,2012-06-07,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208,Before
1761,Taux10ans_O,2012-06-06,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137,Before
1762,Taux10ans_O,2012-06-05,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120,Before
1763,Taux10ans_O,2012-06-04,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905,Before


In [38]:
from datetime import datetime

# Définir la date pivot
date_pivot = datetime(2024, 7, 1)

# S'assurer que la colonne 'Date' est bien convertie en datetime
final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')

# Filtrer les lignes avant la date pivot
filtered_df_before_pivot = final_df[final_df['Date'] < date_pivot]

# Affichage des premières lignes des données avant le 1er juillet 2024
print(filtered_df_before_pivot.head())

# Optionnel : Exporter vers un CSV si besoin
filtered_df_before_pivot.to_csv("avant_1_juillet_2024.csv", index=False)
print("Les données avant le 1er juillet 2024 ont été exportées dans avant_1_juillet_2024.csv")


0  CAC40_O       Date Ouverture Plus haut  Plus bas  \
0  CAC40_O 2013-05-31  3 992,35  3 995,79  3 938,19   
1  CAC40_O 2013-05-30  3 964,50  4 018,97  3 963,69   
2  CAC40_O 2013-05-29  4 024,76  4 028,77  3 970,46   
3  CAC40_O 2013-05-28  4 005,83  4 072,24  4 004,27   
4  CAC40_O 2013-05-27  3 979,15  3 998,05  3 976,41   

0 Fermer      Cours de clôture ajusté en fonction des fractionnements.  \
0                                           3 948,59                     
1                                           3 996,31                     
2                                           3 974,12                     
3                                           4 050,56                     
4                                           3 995,16                     

0 Clôture ajustée      Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.  \
0                                           3 948,59                                        

C:\Users\sarto\AppData\Local\Temp\ipykernel_4940\777239694.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Date'] = pd.to_datetime(final_df['Date'], errors='coerce')


In [39]:
filtered_df_before_pivot

,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonction des fractionnements.,Clôture ajustée Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.,Volume,Avant_Apres_Election
0,CAC40_O,2013-05-31,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800,Before
1,CAC40_O,2013-05-30,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200,Before
2,CAC40_O,2013-05-29,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600,Before
3,CAC40_O,2013-05-28,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800,Before
4,CAC40_O,2013-05-27,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400,Before
...,...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,2012-06-07,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208,Before
1761,Taux10ans_O,2012-06-06,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137,Before
1762,Taux10ans_O,2012-06-05,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120,Before
1763,Taux10ans_O,2012-06-04,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905,Before


In [40]:
erreurs_dates = final_df[final_df['Date'].isna()]
print(erreurs_dates[['Date']].head(10))  # Affiche les 10 premières valeurs problématiques



Empty DataFrame
Columns: [Date]
Index: []


Creating new columns

In [41]:
final_df['Day_of_Week'] = final_df['Date'].dt.day_name()
final_df['Month'] = final_df['Date'].dt.month_name()



C:\Users\sarto\AppData\Local\Temp\ipykernel_4940\1055622838.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Day_of_Week'] = final_df['Date'].dt.day_name()
C:\Users\sarto\AppData\Local\Temp\ipykernel_4940\1055622838.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Month'] = final_df['Date'].dt.month_name()


In [42]:
final_df['Is_Holiday_Period'] = final_df['Month'].apply(lambda x: 'yes' if x in ['November', 'December'] else 'no')


C:\Users\sarto\AppData\Local\Temp\ipykernel_4940\2375034957.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['Is_Holiday_Period'] = final_df['Month'].apply(lambda x: 'yes' if x in ['November', 'December'] else 'no')


In [43]:
final_df

,CAC40_O,Date,Ouverture,Plus haut,Plus bas,Fermer Cours de clôture ajusté en fonction des fractionnements.,Clôture ajustée Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.,Volume,Avant_Apres_Election,Day_of_Week,Month,Is_Holiday_Period
0,CAC40_O,2013-05-31,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800,Before,Friday,May,no
1,CAC40_O,2013-05-30,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200,Before,Thursday,May,no
2,CAC40_O,2013-05-29,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600,Before,Wednesday,May,no
3,CAC40_O,2013-05-28,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800,Before,Tuesday,May,no
4,CAC40_O,2013-05-27,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400,Before,Monday,May,no
...,...,...,...,...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,2012-06-07,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208,Before,Thursday,June,no
1761,Taux10ans_O,2012-06-06,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137,Before,Wednesday,June,no
1762,Taux10ans_O,2012-06-05,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120,Before,Tuesday,June,no
1763,Taux10ans_O,2012-06-04,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905,Before,Monday,June,no


Renaming the columns

In [44]:
print(final_df.columns.to_list())


['CAC40_O', 'Date', 'Ouverture', 'Plus haut', 'Plus bas', 'Fermer      Cours de clôture ajusté en fonction des fractionnements.', 'Clôture ajustée      Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.', 'Volume', 'Avant_Apres_Election', 'Day_of_Week', 'Month', 'Is_Holiday_Period']


In [45]:
final_df.rename(columns={
    'CAC40_O': 'Index',
    'Ouverture': 'Opening',
    'Plus haut': 'Highest',
    'Plus bas': 'Lowest',
    'Fermer      Cours de clôture ajusté en fonction des fractionnements.': 'Closure',
    'Clôture ajustée      Cours de clôture ajusté pour les fractionnements et les distributions de dividendes et/ou de plus-values.': 'Adjusted_Closure',
    'Avant_Apres_Election': 'Before_After_Election'
}, inplace=True)
final_df

C:\Users\sarto\AppData\Local\Temp\ipykernel_4940\2169921842.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={


,Index,Date,Opening,Highest,Lowest,Closure,Adjusted_Closure,Volume,Before_After_Election,Day_of_Week,Month,Is_Holiday_Period
0,CAC40_O,2013-05-31,"3 992,35","3 995,79","3 938,19","3 948,59","3 948,59",133 511 800,Before,Friday,May,no
1,CAC40_O,2013-05-30,"3 964,50","4 018,97","3 963,69","3 996,31","3 996,31",84 607 200,Before,Thursday,May,no
2,CAC40_O,2013-05-29,"4 024,76","4 028,77","3 970,46","3 974,12","3 974,12",91 563 600,Before,Wednesday,May,no
3,CAC40_O,2013-05-28,"4 005,83","4 072,24","4 004,27","4 050,56","4 050,56",99 488 800,Before,Tuesday,May,no
4,CAC40_O,2013-05-27,"3 979,15","3 998,05","3 976,41","3 995,16","3 995,16",39 186 400,Before,Monday,May,no
...,...,...,...,...,...,...,...,...,...,...,...,...
1760,Taux10ans_O,2012-06-07,"134,140625","134,437500","134,062500","134,296875","134,296875",24 208,Before,Thursday,June,no
1761,Taux10ans_O,2012-06-06,"134,671875","134,859375","134,046875","134,187500","134,187500",56 137,Before,Wednesday,June,no
1762,Taux10ans_O,2012-06-05,"135,109375","135,156250","134,625000","134,796875","134,796875",41 120,Before,Tuesday,June,no
1763,Taux10ans_O,2012-06-04,"135,656250","135,843750","134,984375","135,000000","135,000000",55 905,Before,Monday,June,no


Downloading the final df

In [46]:
final_df.to_csv('final_df_O.csv')